In [3]:
from ogb.nodeproppred import PygNodePropPredDataset
dataset = PygNodePropPredDataset(name = 'ogbn-papers100M',root="/hdd1/datasets_gujy/ogb_datasets/dataset/") ## please set the root next time
split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]
# graph = dataset[0]
# graph_coo = graph.edge_index
# print(graph_coo.shape)
dataset

PygNodePropPredDataset()

In [7]:
graph = dataset[0]
print(graph)
graph_coo = graph.edge_index
graph_coo.shape

Data(edge_index=[2, 1615685872], x=[111059956, 128], node_year=[111059956, 1], y=[111059956, 1])


torch.Size([2, 1615685872])

In [8]:
from torch_geometric.utils import to_undirected
un_coo = to_undirected(graph_coo)
print(un_coo.shape)


torch.Size([2, 3228124712])


In [10]:
nodes = graph.y.numpy()
# import numpy as np
# coo = graph_coo.numpy()
print(nodes.shape)
# print(coo.shape)
node_nums = nodes.shape[0]
print(node_nums)
edge_nums = un_coo.shape[1]
print(edge_nums)

(111059956, 1)
111059956
3228124712


In [11]:
# sort the un_direct graph  maybe it is not neccessary
import numpy as np
coo = un_coo.numpy()
coo_sort = coo.T[np.lexsort(coo[0,None])].T
print(coo_sort.shape)
# numpy format had done

(2, 3228124712)


In [12]:
# write the COO
with open('/hdd1/datasets_gujy/ogb_datasets/COO_sorted_file.txt','w') as fp:
    fp.write("nodes_nums:"+str(node_nums)+'--'+ " "+str(edge_nums)+'\n')
    for i in range(edge_nums):
        fp.write(str(coo_sort[0,i])+' '+str(coo_sort[1,i])+' '+str(1)+'\n')
    fp.close()
print("Sorted had done!")
# un_coo_np = un_coo.numpy()
# with open('/hdd1/datasets_gujy/ogb_datasets/COO_file.txt','w') as fp:
#     fp.write("nodes_nums:"+str(node_nums)+'--'+ " "+str(edge_nums)+'\n')
#     for i in range(edge_nums):
#         fp.write(str(coo_sort[0,i])+' '+str(coo_sort[1,i])+' '+str(1)+'\n')
#     fp.close()
## the reason sort is to better count csr

In [14]:
row_index= coo_sort[0,:]
col_index= coo_sort[1,:]
row_offsets = np.zeros([1,node_nums+1]).reshape(-1,)
for i in range(edge_nums):
    row_offsets[row_index[i]+1] += 1
print(row_offsets.shape)
print(row_offsets)
for i in range(1,row_offsets.shape[0]):
    row_offsets[i] = row_offsets[i]+row_offsets[i-1]
print(row_offsets)
## convet to the int data
row_offsets = row_offsets.astype(int)
print(row_offsets)

(169344,)


array([  0., 291.,   2., ...,  18.,  20.,   2.])

In [18]:
# save CSR related txt
col = col_index.tolist()
rf = row_offsets.tolist()

with open('/hdd1/datasets_gujy/ogb_datasets/CSR/row_offsets.txt','w') as fp:
    fp.write("node_nums:"+str(node_nums)+';'+ " edge_nums: "+str(edge_nums)+" the value is 1 to present connect; attention: the length of row offsets is node_length + 1"+'\n')
    [fp.write(str(item)+'\n') for  item in rf]
    fp.close()
with open('/hdd1/datasets_gujy/ogb_datasets/CSR/colum_index.txt','w') as fp:
    fp.write("node_nums:"+str(node_nums)+'--'+ " edge_nums: "+str(edge_nums)+" the value is 1 to present connect "+'\n')
    [fp.write(str(item)+' '+str(1)+'\n') for item in col]
    fp.close()